In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, train_test_split
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

wsize=40

def pre_process(path, cut_off=wsize):
    col_names=['seqid', 'wstart','wend','ID']+list(range(80))
    c_cols=list(range(80))
    df=pd.read_csv(path, sep='\t', header=None, names=col_names)
    sl=df[df['ID'].str.contains('SL')].reset_index(drop=True)
    el=df[df['ID'].str.contains('EL')].reset_index(drop=True)
    intron=df[df['ID'].str.contains('intron')].reset_index(drop=True)
    exon=df[df['ID'].str.contains('exon')].reset_index(drop=True)
    rev_idx= ['seqid','wstart','wend','ID']+list(range(79,-1,-1))
    sl_rev=sl.loc[:,rev_idx]
    sl_rev.columns=col_names
    all_junc=pd.concat([el, sl_rev]).reset_index(drop=True)
    keep_junc=all_junc.loc[:, c_cols].sum(axis=1) >=80
    all_junc=all_junc[keep_junc]
    keep_exon=exon.loc[:,c_cols].sum(axis=1) >=80
    exon=exon[keep_exon]
    intron_sums=intron.loc[:,c_cols].sum(axis=1)
    intron_sums.quantile([.1,.2,.3,.4,.6,.7,.8,.9, 1])# majority are 0, but i'll keep it up till 20 for sake of completeness.
    keep_intron=intron_sums <= 20
    intron=intron[keep_intron]

    min_count= min(all_junc.ID.str.contains('ref').sum(),
                  all_junc.shape[0]-all_junc.ID.str.contains('ref').sum() ,
                  intron.shape[0],
                  exon.shape[0])
    complete_data=pd.concat([all_junc[all_junc['ID'].str.contains('ref')].sample(min_count, ),
                             all_junc[~all_junc['ID'].str.contains('ref')].sample(min_count),
                             intron.sample(min_count),
                             exon.sample(min_count)
                            ]).reset_index(drop=True)

    lab_col=list()
    '''
    0=alt_splice junctino
    1=no_splice junction
    2=exon
    3=not exon
    '''
    for lab in complete_data['ID']:
        if 'intron' in lab:
            lab_col.append(3)
        elif 'exon' in lab:
            lab_col.append(2)
        elif 'ref' in lab:
            lab_col.append(1)
        else:
            lab_col.append(0)

    complete_data['Y']=lab_col
    return(complete_data)


os.chdir('/Users/vinayswamy/NIH/eyesplice_predictor/')


In [2]:
data=pre_process('data/cleaned_cov/RPE_Fetal.Tissue/HM7FMBBXX_16424750_S70_bp_features.tsv.gz')
data_junc=data.query('Y == 0 | Y == 1')
x_cols= list(range(80))
X=data_junc.loc[:,x_cols]
Y=data_junc['Y']
X_train, X_test , Y_train, Y_test= train_test_split(X, Y,test_size=.3, random_state=343434) 

In [15]:
'''

learning_rate =0.1,
n_estimators=1000,
max_depth=4,
min_child_weight=6,
gamma=0,
subsample=0.8,
colsample_bytree=0.8,
reg_alpha=0.005,
reg_lambda
objective= 'binary:logistic',
nthread=4,
scale_pos_weight=1,

base
learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree



'''



def opti_wrapper_full(n_learning_rate, n_nestimators, n_maxdepth, n_minchild, n_gamme, n_subsample,
                 n_colsample, n_regalpha, n_reglambda, n_scalePosWeight):
    model= XGBClassifier(learning_rate=n_learning_rate, n_estimators=n_nestimators, 
                         max_depth=n_maxdepth, min_child_weight=n_minchild, gamma=n_gamme, 
                         subsample=n_subsample, colsample_bytree=n_colsample, 
                         reg_alpha=n_regalpha, reg_lambda=n_reglambda, 
                         scale_pos_weight=n_scalePosWeight, random_state=223232)
    score= cross_val_score(model, X_test,Y_test)
    return(score)


def opti_wrapper_base(n_learning_rate,  n_gamme,n_colsample, n_regalpha):
    model= XGBClassifier(learning_rate=n_learning_rate, gamma=n_gamme,colsample_bytree=n_colsample, 
                         reg_alpha=n_regalpha,random_state=223232)
    score= cross_val_score(model, X_train,Y_train, cv=3).mean()
    return(score)
param_bounds={'n_learning_rate' : (.01, .3),
              #'n_maxdepth' : (3,15),
              #'n_minchild' : (1,10), 
              'n_gamme' : (0,.5),
              'n_colsample' : (.3,.8 ), 
              'n_regalpha' : (1e-5, 100)
             }


In [16]:
from bayes_opt import BayesianOptimization
optimizer=BayesianOptimization(f=opti_wrapper_base, pbounds=param_bounds, random_state=1234, verbose=2)

In [17]:
params=optimizer.maximize(init_points=5, n_iter=10)

|   iter    |  target   | n_cols... |  n_gamme  | n_lear... | n_rega... |
-------------------------------------------------------------------------
|  1        |  0.7658   |  0.3958   |  0.3111   |  0.1369   |  78.54    |
|  2        |  0.7667   |  0.69     |  0.1363   |  0.09017  |  80.19    |
|  3        |  0.7673   |  0.7791   |  0.438    |  0.1138   |  50.1     |
|  4        |  0.7667   |  0.6417   |  0.3564   |  0.1174   |  56.12    |
|  5        |  0.7661   |  0.5515   |  0.006884 |  0.2341   |  88.26    |
|  6        |  0.7682   |  0.5938   |  0.1764   |  0.09206  |  0.000290 |
|  7        |  0.7645   |  0.7378   |  0.3999   |  0.2938   |  0.000107 |
|  8        |  0.7671   |  0.6334   |  0.2062   |  0.01229  |  0.009184 |
|  9        |  0.7659   |  0.3478   |  0.4117   |  0.1922   |  0.002077 |
|  10       |  0.7654   |  0.3382   |  0.2541   |  0.108    |  100.0    |
|  11       |  0.767    |  0.3104   |  0.3591   |  0.01921  |  0.01143  |
|  12       |  0.7677   |  0.4868   | 

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix , roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from xgboost import XGBClassifier
import pandas_ml as pml

def ROC_plot(Y_test, Y_prob):
    fpr, tpr, thresholds=roc_curve(Y_test, Y_prob)
    auc=roc_auc_score(Y_test, Y_prob)
    # Plot ROC curve
    plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % auc)
    plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate or (1 - Specifity)')
    plt.ylabel('True Positive Rate or (Sensitivity)')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()
def PR_plot(Y_test, Y_prob):
    pre, rec, thresholds = precision_recall_curve(Y_test, Y_prob)
    auc = average_precision_score(Y_test, Y_prob)
    plt.plot(rec, pre, label=' Prec/Rec (area = %0.2f)' % ( auc))
    plt.plot([1, 1], [1, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('')
    plt.legend(loc="lower right")
    plt.show()   

def train_model(X,Y, model):
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=.3, random_state=8976, stratify=Y)
    model.fit(X_train, Y_train)
    Y_pred=model.predict(X_test)
    Y_prob=model.predict_proba(X_test)[:,1]
    labs=model.classes_
    print('confusion matrix\n\n')
    print(pd.DataFrame(confusion_matrix(Y_test, Y_pred), index=labs, columns=labs))
    print('\n\nclassification report\n\n')
    print(classification_report(y_pred=Y_pred,y_true=Y_test))
    #ROC_plot(Y_test , Y_prob)
    #PR_plot(Y_test,Y_prob)
    return(model)



In [19]:
x_cols=list(range(80))
model_vanilla=XGBClassifier(random_state=2234)
trained_model= train_model(X, Y, model_vanilla)

confusion matrix


      0     1
0  4815  1632
1  1380  5067


classification report


              precision    recall  f1-score   support

           0       0.78      0.75      0.76      6447
           1       0.76      0.79      0.77      6447

   micro avg       0.77      0.77      0.77     12894
   macro avg       0.77      0.77      0.77     12894
weighted avg       0.77      0.77      0.77     12894



In [20]:
model_opti=XGBClassifier(colsample_bytree=.5938, gamma=.1764, learning_rate=.09206, reg_alpha=.000290, random_state=343434)
tm_opti=train_model(X,Y, model_opti)

confusion matrix


      0     1
0  4817  1630
1  1378  5069


classification report


              precision    recall  f1-score   support

           0       0.78      0.75      0.76      6447
           1       0.76      0.79      0.77      6447

   micro avg       0.77      0.77      0.77     12894
   macro avg       0.77      0.77      0.77     12894
weighted avg       0.77      0.77      0.77     12894

